In [2]:
import pandas as pd
from sodapy import Socrata
import os
import plotly.plotly as py
import plotly.figure_factory as ff
from plotly.graph_objs import *

In [98]:
# Get Graffiti 311 Data from Chicago Data Portal API
graffiti_id = "cdmx-wzbz"
client = Socrata("data.cityofchicago.org", 'BOetEt3G4uynCIa1XQuFyRS8X')

g_results = client.get(graffiti_id, where="creation_date between '2017-01-01T00:00:00' and '2017-12-31T00:00:00'", limit = 2000000)
g_df = pd.DataFrame.from_records(g_results)
g_df.drop(["what_type_of_surface_is_the_graffiti_on_", "where_is_the_graffiti_located_", "ssa"], axis = 1, inplace = True)


In [99]:
g_df.shape

(112779, 15)

In [100]:
# Get Vacant and Abandoned Buildings 311 Data from Chicago Data Portal API
buildings_id = "yama-9had"
b_results = client.get(buildings_id, where="date_service_request_was_received between '2017-01-01T00:00:00' and '2017-12-31T00:00:00'", limit = 2000000)
b_df = pd.DataFrame.from_records(b_results)
b_df["street_address"] = b_df.address_street_number+" "+b_df.address_street_direction+" "+b_df.address_street_name+" "+b_df.address_street_suffix
b_df.drop(["location_of_building_on_the_lot_if_garage_change_type_code_to_bgd_", "is_building_open_or_boarded_", 
           "if_the_building_is_open_where_is_the_entry_point_", "is_the_building_currently_vacant_or_occupied_",
           "is_the_building_vacant_due_to_fire_", "any_people_using_property_homeless_childen_gangs_", "address_street_number",
           "address_street_direction", "address_street_name", "address_street_suffix"], axis = 1, inplace = True)
b_df["completion_date"] = None
b_df.rename(index=str, columns={"date_service_request_was_received": "creation_date", "service_request_type": "type_of_service_request"}, inplace = True)


In [101]:
# Get Alley Lights Out 311 Data from Chicago Data Portal API
alleys_id = "j9pw-ad5p"
a_results = client.get(alleys_id, where="creation_date between '2017-01-01T00:00:00' and '2017-12-31T00:00:00'", limit = 2000000)
a_df = pd.DataFrame.from_records(a_results)

In [111]:
com_area_id = "igwz-8jzy"
ca_results = client.get(com_area_id)
ca_df = pd.DataFrame.from_records(ca_results)

In [119]:
ca_names = ca_df[["area_num_1", "community"]].copy()
ca_names.rename(index = str, columns = {"area_num_1": "community_area"}, inplace = True)

In [121]:
# Concatenate the 3 dataframes into one dataframe representing all 311 calls in 2017 on the 3 issues
all_call = pd.concat([g_df, b_df, a_df], axis = 0)
all_call = pd.merge(all_call, ca_names, on = 'community_area', how = 'left')
all_call['response_time'] = pd.to_datetime(all_call['completion_date']) - pd.to_datetime(all_call['creation_date'])

In [131]:
all_call['response_time'] = (pd.to_datetime(all_call['completion_date']) - pd.to_datetime(all_call['creation_date'])).dt.days
all_call[:10]

,community_area,completion_date,creation_date,latitude,location,longitude,police_district,service_request_number,status,street_address,type_of_service_request,ward,x_coordinate,y_coordinate,zip_code,community,response_time
0,66,2017-08-07T00:00:00.000,2017-08-07T00:00:00.000,41.76386983154277,"{'type': 'Point', 'coordinates': [-87.68674560...",-87.68674560999156,8,17-05251065,Completed,7125 S MAPLEWOOD AVE,Graffiti Removal,18,1160606.3195708,1857319.79946719,60629,CHICAGO LAWN,0.0
1,4,2017-08-07T00:00:00.000,2017-08-05T00:00:00.000,41.96162484771538,"{'type': 'Point', 'coordinates': [-87.68976961...",-87.68976961706613,19,17-05196842,Completed,4409 N ARTESIAN AVE,Graffiti Removal,47,1159245.97991312,1929206.66469635,60625,LINCOLN SQUARE,2.0
2,28,2017-08-07T00:00:00.000,2017-08-07T00:00:00.000,41.86497176596717,"{'type': 'Point', 'coordinates': [-87.66623577...",-87.66623577475536,12,17-05248568,Completed,1557 W 13TH ST,Graffiti Removal,28,1166057.81087376,1894135.42000548,60608,NEAR WEST SIDE,0.0
3,21,2017-08-07T00:00:00.000,2017-08-07T00:00:00.000,41.94515063717326,"{'type': 'Point', 'coordinates': [-87.70793142...",-87.70793142018661,17,17-05246644,Completed,3510 N KEDZIE AVE,Graffiti Removal,35,1154428.449525,1923227.687975,60618,AVONDALE,0.0
4,4,2017-08-07T00:00:00.000,2017-08-05T00:00:00.000,41.96306063501604,"{'type': 'Point', 'coordinates': [-87.68636754...",-87.68636754917809,19,17-05201961,Completed - Dup,2301 W SUNNYSIDE AVE,Graffiti Removal,47,1160210.11016623,1929833.54508122,60625,LINCOLN SQUARE,2.0
5,20,2017-08-07T00:00:00.000,2017-08-05T00:00:00.000,41.92705533134073,"{'type': 'Point', 'coordinates': [-87.73923689...",-87.73923689278473,25,17-05194517,Completed,2520 N KILBOURN AVE,Graffiti Removal,31,1145958.36982059,1916551.22010653,60639,HERMOSA,2.0
6,22,2017-08-07T00:00:00.000,2017-08-05T00:00:00.000,41.91512917985401,"{'type': 'Point', 'coordinates': [-87.68526200...",-87.68526200202442,14,17-05197380,Completed,2306 W MOFFAT ST,Graffiti Removal,32,1160680.44779372,1912316.97737679,60647,LOGAN SQUARE,2.0
7,56,2017-08-07T00:00:00.000,2017-08-05T00:00:00.000,41.78838364548609,"{'type': 'Point', 'coordinates': [-87.79305432...",-87.79305432924627,8,17-05203102,Completed,6841 W 57TH ST,Graffiti Removal,13,1131653.63100375,1865982.0165813,60638,GARFIELD RIDGE,2.0
8,56,2017-08-07T00:00:00.000,2017-08-07T00:00:00.000,41.80018308981581,"{'type': 'Point', 'coordinates': [-87.74280941...",-87.74280941209145,8,17-05249942,Completed,5103 S CICERO AVE,Graffiti Removal,14,1145222.56478353,1870370.20420231,60632,GARFIELD RIDGE,0.0
9,59,2017-08-07T00:00:00.000,2017-08-07T00:00:00.000,41.82745823204983,"{'type': 'Point', 'coordinates': [-87.67505698...",-87.6750569817486,9,17-05245737,Completed,3637 S DAMEN AVE,Graffiti Removal,12,1163616.43658405,1880558.68722658,60609,MCKINLEY PARK,0.0


In [138]:
# Count number of service requests by type
all_call.groupby("type_of_service_request").size()

type_of_service_request
Alley Light Out               27896
Graffiti Removal             112779
Vacant/Abandoned Building      3666
dtype: int64

In [137]:
# Count number of service requests by community area
grouped = all_call.groupby("community").size()
grouped.sort_values(ascending=False)

community
WEST TOWN             10272
LOGAN SQUARE           8162
SOUTH LAWNDALE         6675
LOWER WEST SIDE        6216
BRIGHTON PARK          5784
AVONDALE               4751
LAKE VIEW              4730
MCKINLEY PARK          4693
GAGE PARK              3825
BELMONT CRAGIN         3744
IRVING PARK            3706
NORTH CENTER           3638
LINCOLN SQUARE         3582
NEAR WEST SIDE         3373
LINCOLN PARK           3245
PORTAGE PARK           3053
ALBANY PARK            3017
CHICAGO LAWN           2860
WEST ELSDON            2713
HUMBOLDT PARK          2638
AUSTIN                 2506
WEST LAWN              2497
NEW CITY               2461
ARCHER HEIGHTS         2455
LOOP                   2387
BRIDGEPORT             2248
ASHBURN                2151
UPTOWN                 2018
NEAR NORTH SIDE        1974
WEST RIDGE             1913
                      ...  
WASHINGTON HEIGHTS      757
JEFFERSON PARK          724
ARMOUR SQUARE           699
WEST GARFIELD PARK      645
EAST GARFI

In [134]:
grouped = all_call.groupby("community")['response_time'].mean()
grouped.sort_values(ascending=False)

community
OAKLAND                   43.829787
RIVERDALE                 37.230769
MORGAN PARK               34.898876
WEST PULLMAN              32.679831
BEVERLY                   31.305000
GREATER GRAND CROSSING    30.977099
CHATHAM                   30.262887
AVALON PARK               30.225490
WOODLAWN                  29.456929
SOUTH SHORE               28.497592
AUBURN GRESHAM            28.209302
KENWOOD                   27.666667
ENGLEWOOD                 26.647799
CALUMET HEIGHTS           26.448000
WASHINGTON HEIGHTS        24.181159
PULLMAN                   23.336066
WEST GARFIELD PARK        23.310493
WEST ENGLEWOOD            22.235686
MOUNT GREENWOOD           22.189964
FULLER PARK               21.649485
ROSELAND                  21.307377
HEGEWISCH                 21.178439
EDISON PARK               20.782946
WASHINGTON PARK           20.247706
GRAND BOULEVARD           19.981191
SOUTH DEERING             19.849693
SOUTH CHICAGO             18.500876
NORWOOD PARK      

In [136]:
grouped = all_call.groupby("type_of_service_request")['response_time'].mean()
grouped.sort_values(ascending=False)

type_of_service_request
Alley Light Out              33.405957
Graffiti Removal              0.676226
Vacant/Abandoned Building          NaN
Name: response_time, dtype: float64

The findings below illuminate several interesting trends about responses to 311 requests in the city of Chicago. I highlight five interesting things I learned performing this analysis below:
1. The city of Chicago receives vastly more graffiti removal requests than alley lights out requests or vacant building requests. This result likely indicates the higher incidence of graffiti than vacant buildings or alley lights out. 
2. There are significant disparities in the average response time to 311 calls in the city. The areas that experience the fastest response times tend to be of key commercial importance (such as the Loop or O'Hare Airport) or higher-income areas. The areas with the slowest response times tend to be lower-income areas.
3. There is a significant disconnect between the community areas that place the most 311 requests and the response time to 311 requests in those areas. This may reflect that citizens living in community areas with slow response times have become disenfranchised and do not expect the city to respond to their needs. 
4.
5.